<a href="https://colab.research.google.com/github/ranzatu4256/LLM_gameAgent/blob/main/LLM_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *偵察兵*
長距離の移動・探知が可能
*   移動可能距離 : 3マス
*   探知距離 : 4マス
*   攻撃距離 : 3マス
*   通信可能距離 : 3マス

In [ ]:
class Scout:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.hp = 5
        self.attack_power = 1
        self.move_range = 3
        self.scout_range = 4
        self.attack_range = 3
        self.inform_range = 2
        self.known_enemies = set()
        self.mobility_policy = ""
        self.attack_policy = ""
        self.symbol = 'S' if self.team == 'ally' else 's'

    def move(self, new_x, new_y, board):
        if self.is_valid_move(new_x, new_y, board):
            board[self.y][self.x] = '.'
            self.x, self.y = new_x, new_y
            board[self.y][self.x] = 'S' if self.team == 'ally' else 's'

    def is_valid_move(self, new_x, new_y, board):
        return (0 <= new_x < len(board) and 0 <= new_y < len(board) and
                abs(new_x - self.x) + abs(new_y - self.y) <= self.move_range and
                board[new_y][new_x] == '.')

    def scout(self, board, pieces):
        for dy in range(-self.scout_range, self.scout_range + 1):
            for dx in range(-self.scout_range, self.scout_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.scout_range:
                    if (self.team == 'ally' and board[new_y][new_x].islower()) or \
                       (self.team == 'enemy' and board[new_y][new_x].isupper()):
                        for piece in pieces:
                            if piece.x == new_x and piece.y == new_y:
                                self.known_enemies.add(piece)

    def inform_allies(self, board, enemy_positions):
        for dy in range(-self.inform_range, self.inform_range + 1):
            for dx in range(-self.inform_range, self.inform_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.inform_range:
                    if (self.team == 'ally' and board[new_y][new_x].isupper()) or \
                       (self.team == 'enemy' and board[new_y][new_x].islower()):
                        print(f"Informing ally at ({new_x}, {new_y}) about enemy positions: {enemy_positions}")

    def receive_information(self, sender, enemy_positions):
        if abs(sender.x - self.x) + abs(sender.y - self.y) <= sender.inform_range:
            self.known_enemies.update(enemy_positions)

    def attack(self, target_x, target_y, board):
        if self.is_valid_attack(target_x, target_y, board):
            print(f"Scout attacks position ({target_x}, {target_y})")
            # ここで攻撃対象の駒のHPを減らすなどの処理を行う

    def is_valid_attack(self, target_x, target_y, board):
        dx = abs(target_x - self.x)
        dy = abs(target_y - self.y)
        return (0 <= target_x < len(board) and 0 <= target_y < len(board) and
                #max(dx, dy) <= self.attack_range and
                abs(target_x - self.x) + abs(target_y - self.y) <= self.attack_range and
                ((self.team == 'ally' and board[target_y][target_x].islower()) or
                 (self.team == 'enemy' and board[target_y][target_x].isupper())))

# *衛生兵*
HPが0となった味方の蘇生が可能

*  移動可能距離 : 2マス
*  探知距離 : 3マス
*  攻撃距離 : 3マス
*  通信可能距離 : 3マス
*  蘇生可能距離 : 3マス

In [ ]:
class Medic:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.hp = 5
        self.attack_power = 1
        self.move_range = 2
        self.scout_range = 3
        self.attack_range = 3
        self.inform_range = 3
        self.revive_range = 3
        self.known_enemies = set()
        self.mobility_policy = ""
        self.attack_policy = ""
        self.symbol = 'M' if self.team == 'ally' else 'm'

    def move(self, new_x, new_y, board):
        if self.is_valid_move(new_x, new_y, board):
            board[self.y][self.x] = '.'
            self.x, self.y = new_x, new_y
            board[self.y][self.x] = 'S' if self.team == 'ally' else 's'

    def is_valid_move(self, new_x, new_y, board):
        return (0 <= new_x < len(board) and 0 <= new_y < len(board) and
                abs(new_x - self.x) + abs(new_y - self.y) <= self.move_range and
                board[new_y][new_x] == '.')

    def scout(self, board, pieces):
        for dy in range(-self.scout_range, self.scout_range + 1):
            for dx in range(-self.scout_range, self.scout_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.scout_range:
                    if (self.team == 'ally' and board[new_y][new_x].islower()) or \
                       (self.team == 'enemy' and board[new_y][new_x].isupper()):
                        for piece in pieces:
                            if piece.x == new_x and piece.y == new_y:
                                self.known_enemies.add(piece)

    def inform_allies(self, board, enemy_positions):
        for dy in range(-self.inform_range, self.inform_range + 1):
            for dx in range(-self.inform_range, self.inform_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.inform_range:
                    if (self.team == 'ally' and board[new_y][new_x].isupper()) or \
                       (self.team == 'enemy' and board[new_y][new_x].islower()):
                        print(f"Informing ally at ({new_x}, {new_y}) about enemy positions: {enemy_positions}")

    def receive_information(self, sender, enemy_positions):
        if abs(sender.x - self.x) + abs(sender.y - self.y) <= sender.inform_range:
            self.known_enemies.update(enemy_positions)

    def revive(self, board, defeated_pieces):
        for piece in defeated_pieces:
            if piece.team == self.team and (piece.symbol != "m" or piece.symbol != "M"):
                if abs(piece.x - self.x) + abs(piece.y - self.y) <= self.revive_range:
                    piece.hp = 1  # 蘇生時のHP
                    defeated_pieces.remove(piece)
                    print(f"{self.symbol} at ({self.x}, {self.y}) revived {piece.symbol} at ({piece.x}, {piece.y})")
        return defeated_pieces

    def attack(self, target_x, target_y, board):
        if self.is_valid_attack(target_x, target_y, board):
            print(f"Medic attacks position ({target_x}, {target_y})")
            # ここで攻撃対象の駒のHPを減らすなどの処理を行う

    def is_valid_attack(self, target_x, target_y, board):
        dx = abs(target_x - self.x)
        dy = abs(target_y - self.y)
        return (0 <= target_x < len(board) and 0 <= target_y < len(board) and
                #max(dx, dy) <= self.attack_range and
                abs(target_x - self.x) + abs(target_y - self.y) <= self.attack_range and
                ((self.team == 'ally' and board[target_y][target_x].islower()) or
                 (self.team == 'enemy' and board[target_y][target_x].isupper())))

# 重装備兵
移動距離が短い代わりに、長距離・広範囲・高火力の攻撃が可能
*  移動可能距離 : 1マス
*  探知距離 : 3マス
*  攻撃距離 : 4マス
*  通信可能距離 : 3マス
*  攻撃範囲 : 周辺1マス

In [ ]:
class HeavyInfantry:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.hp = 5
        self.attack_power = 3
        self.move_range = 1
        self.scout_range = 3
        self.attack_range = 4
        self.inform_range = 3
        self.attack_area = 1
        self.known_enemies = set()
        self.mobility_policy = ""
        self.attack_policy = ""
        self.symbol = 'H' if self.team == 'ally' else 'h'

    def move(self, new_x, new_y, board):
        if self.is_valid_move(new_x, new_y, board):
            board[self.y][self.x] = '.'
            self.x, self.y = new_x, new_y
            board[self.y][self.x] = 'S' if self.team == 'ally' else 's'

    def is_valid_move(self, new_x, new_y, board):
        return (0 <= new_x < len(board) and 0 <= new_y < len(board) and
                abs(new_x - self.x) + abs(new_y - self.y) <= self.move_range and
                board[new_y][new_x] == '.')

    def scout(self, board, pieces):
        for dy in range(-self.scout_range, self.scout_range + 1):
            for dx in range(-self.scout_range, self.scout_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.scout_range:
                    if (self.team == 'ally' and board[new_y][new_x].islower()) or \
                       (self.team == 'enemy' and board[new_y][new_x].isupper()):
                        for piece in pieces:
                            if piece.x == new_x and piece.y == new_y:
                                self.known_enemies.add(piece)

    def inform_allies(self, board, enemy_positions):
        for dy in range(-self.inform_range, self.inform_range + 1):
            for dx in range(-self.inform_range, self.inform_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.inform_range:
                    if (self.team == 'ally' and board[new_y][new_x].isupper()) or \
                       (self.team == 'enemy' and board[new_y][new_x].islower()):
                        print(f"Informing ally at ({new_x}, {new_y}) about enemy positions: {enemy_positions}")

    def receive_information(self, sender, enemy_positions):
        if abs(sender.x - self.x) + abs(sender.y - self.y) <= sender.inform_range:
            self.known_enemies.update(enemy_positions)

    def attack(self, target_x, target_y, board):
        if self.is_valid_attack(target_x, target_y):
            print(f"Heavy Infantry attacks position ({target_x}, {target_y}) and surrounding area")
            affected_positions = self.get_attack_area(target_x, target_y)
            for pos_x, pos_y in affected_positions:
                if 0 <= pos_x < 9 and 0 <= pos_y < 9:
                    if (self.team == 'ally' and board[pos_y][pos_x].islower()) or \
                       (self.team == 'enemy' and board[pos_y][pos_x].isupper()):
                        print(f"Attacking enemy at ({pos_x}, {pos_y})")
                        # ここで攻撃対象の駒のHPを減らすなどの処理を行う

    def is_valid_attack(self, target_x, target_y, board):
        dx = abs(target_x - self.x)
        dy = abs(target_y - self.y)
        return (0 <= target_x < len(board) and 0 <= target_y < len(board) and
                #max(dx, dy) <= self.attack_range and
                abs(target_x - self.x) + abs(target_y - self.y) <= self.attack_range and
                ((self.team == 'ally' and board[target_y][target_x].islower()) or
                 (self.team == 'enemy' and board[target_y][target_x].isupper())))

    def get_attack_area(self, center_x, center_y):
        area = []
        for dy in range(-self.attack_area, self.attack_area + 1):
            for dx in range(-self.attack_area, self.attack_area + 1):
                area.append((center_x + dx, center_y + dy))
        return area

# 通信兵
広範囲の味方と情報共有が可能
*  移動可能距離 : 2マス
*  探知距離 : 3マス
*  攻撃距離 : 3マス
*  通信可能距離 : 4マス

In [ ]:
class Communicator:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.team = team
        self.hp = 5
        self.attack_power = 1
        self.move_range = 2
        self.scout_range = 3
        self.attack_range = 3
        self.inform_range = 4
        self.known_enemies = set()
        self.mobility_policy = ""
        self.attack_policy = ""
        self.symbol = 'C' if self.team == 'ally' else 'c'

    def move(self, new_x, new_y, board):
        if self.is_valid_move(new_x, new_y, board):
            board[self.y][self.x] = '.'
            self.x, self.y = new_x, new_y
            board[self.y][self.x] = 'S' if self.team == 'ally' else 's'

    def is_valid_move(self, new_x, new_y, board):
        return (0 <= new_x < len(board) and 0 <= new_y < len(board) and
                abs(new_x - self.x) + abs(new_y - self.y) <= self.move_range and
                board[new_y][new_x] == '.')

    def scout(self, board, pieces):
        for dy in range(-self.scout_range, self.scout_range + 1):
            for dx in range(-self.scout_range, self.scout_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.scout_range:
                    if (self.team == 'ally' and board[new_y][new_x].islower()) or \
                       (self.team == 'enemy' and board[new_y][new_x].isupper()):
                        for piece in pieces:
                            if piece.x == new_x and piece.y == new_y:
                                self.known_enemies.add(piece)

    def inform_allies(self, board, enemy_positions):
        for dy in range(-self.inform_range, self.inform_range + 1):
            for dx in range(-self.inform_range, self.inform_range + 1):
                new_x, new_y = self.x + dx, self.y + dy
                if 0 <= new_x < 9 and 0 <= new_y < 9 and abs(new_x - self.x) + abs(new_y - self.y) <= self.inform_range:
                    if (self.team == 'ally' and board[new_y][new_x].isupper()) or \
                       (self.team == 'enemy' and board[new_y][new_x].islower()):
                        print(f"Informing ally at ({new_x}, {new_y}) about enemy positions: {enemy_positions}")

    def receive_information(self, sender, enemy_positions):
        if abs(sender.x - self.x) + abs(sender.y - self.y) <= sender.inform_range:
            self.known_enemies.update(enemy_positions)

    def attack(self, target_x, target_y, board):
        if self.is_valid_attack(target_x, target_y, board):
            print(f"Communicator attacks position ({target_x}, {target_y})")
            # ここで攻撃対象の駒のHPを減らすなどの処理を行う

    def is_valid_attack(self, target_x, target_y, board):
        dx = abs(target_x - self.x)
        dy = abs(target_y - self.y)
        return (0 <= target_x < len(board) and 0 <= target_y < len(board) and
                #max(dx, dy) <= self.attack_range and
                abs(target_x - self.x) + abs(target_y - self.y) <= self.attack_range and
                ((self.team == 'ally' and board[target_y][target_x].islower()) or
                 (self.team == 'enemy' and board[target_y][target_x].isupper())))

# ゲーム
勝利条件
*  敵の全滅
*  最奥のマスに到達
*  10ターン経過後、合計HPが高い

In [ ]:
import random

class Game:
    def __init__(self, board_size=9):
        self.board_size = board_size
        self.board = [['.' for _ in range(board_size)] for _ in range(board_size)]
        self.pieces = []
        self.defeated_pieces = []
        self.current_turn = 0
        self.max_turns = 10
        self.player_a = None
        self.player_b = None

    def add_piece(self, piece):
        self.pieces.append(piece)
        self.board[piece.y][piece.x] = piece.symbol

    def remove_piece(self, piece):
        self.pieces.remove(piece)
        self.board[piece.y][piece.x] = '.'

    def move_piece(self, piece, new_x, new_y):
        if piece.is_valid_move(new_x, new_y, self.board):
            self.board[piece.y][piece.x] = '.'
            piece.x, piece.y = new_x, new_y
            self.board[new_y][new_x] = piece.symbol

    def play_turn(self):
        self.current_turn += 1
        print(f"\n--- Turn {self.current_turn} ---")

        # 1. 自軍の駒の移動
        self.move_team_pieces('ally')

        # 2. 敵軍の駒の移動
        self.move_team_pieces('enemy')

        self.print_board()
        # 3. 情報共有
        self.share_information()

        # 4. 自軍の攻撃対象選択
        ally_attacks = self.select_attack_targets('ally')

        # 5. 敵軍の攻撃対象選択
        enemy_attacks = self.select_attack_targets('enemy')

        # 6. ダメージ計算
        self.calculate_damage(ally_attacks + enemy_attacks)

        # 7. 蘇生フェーズ
        self.revive_phase()

        # 8. ターン終了処理
        self.end_turn()

    def set_pieces_policy(self, team):
        if team == 'ally':
            self.set_pieces_policy_cli(self.player_a, team)
        else:
            self.set_pieces_policy_cli(self.player_b, team)

    def set_pieces_policy_cli(self, player, team):
        for piece in self.pieces:
            if piece.team == team:
                piece.mobility_policy = input(f"Input the mobility polisy of {piece.symbol}.")
                if piece.mobility_policy == '0':
                    piece.mobility_policy = "You basically follow the policy of moving forward."
                piece.attack_policy = input(f"Input the attack polisy of {piece.symbol}.")
                if piece.attack_policy == '0':
                    piece.attack_policy = "You basically follow the policy of attacking enemies in the forward direction."


    def move_team_pieces(self, team):
        if team == 'ally':
            self.move_pieces_cli(self.player_a, team)
        else:
            self.move_pieces_cli(self.player_b, team)

    def move_pieces_cli(self, player, team):
        for piece in self.pieces:
            if piece.team == team:
                possible_moves = [
                    (dx, dy)
                    for dx in range(-piece.move_range, piece.move_range + 1)
                    for dy in range(-piece.move_range, piece.move_range + 1)
                    #if piece.is_valid_move(piece.x + dx, piece.y + dy, self.board)
                    if abs(dx) + abs(dy) <= piece.move_range and piece.is_valid_move(piece.x + dx, piece.y + dy, self.board)
                ]
                possible_moves.append((0, 0))  # 動かないオプションを追加
                dx, dy = player.select_move(piece, possible_moves)
                new_x, new_y = piece.x + dx, piece.y + dy
                self.move_piece(piece, new_x, new_y)
                print(f"{piece.symbol} moved to ({new_x}, {new_y})")

    def share_information(self):
        # まず、全ての駒が周囲をスカウトします
        for piece in self.pieces:
            piece.known_enemies = set()
            piece.scout(self.board, self.pieces)

        # 次に、情報共有を行います
        for sender in self.pieces:
            for receiver in self.pieces:
                if sender != receiver and sender.team == receiver.team:
                    receiver.receive_information(sender, sender.known_enemies)
        for sender in reversed(self.pieces):
            for receiver in self.pieces:
                if sender != receiver and sender.team == receiver.team:
                    receiver.receive_information(sender, sender.known_enemies)

    def select_attack_targets(self, team):
        if team == 'ally':
            return self.select_attack_targets_cli(self.player_a, team)
        else:
            return self.select_attack_targets_cli(self.player_b, team)

    def select_attack_targets_cli(self, player, team):
        attacks = []
        for attacker in self.pieces:
            if attacker.team == team:
                targets = []
                for target in self.pieces:
                    if target.team != attacker.team:
                        dx = abs(target.x - attacker.x)
                        dy = abs(target.y - attacker.y)
                        if dx+dy <= attacker.attack_range:
                            targets.append(target)
                targets = list(set(targets) & attacker.known_enemies)
                print(f"{attacker.symbol} knows the following targets:")
                for enemy in  attacker.known_enemies:
                    print(f"-{enemy.symbol} at ({enemy.x}, {enemy.y})")
                print(f"{attacker.symbol} can attack the following targets:")
                for target in targets:
                    print(f"-{target.symbol} at ({target.x}, {target.y})")
                if targets:
                    target = player.select_attack_target(attacker, targets)
                    attacks.append((attacker, target))
        return attacks

    def calculate_damage(self, attacks):
        self.defeated_pieces = []
        for attacker, target in attacks:
            damage = attacker.attack_power
            target.hp -= damage
            print(f"{attacker.symbol} attacks {target.symbol} for {damage} damage. {target.symbol}'s HP: {target.hp}")
            if target.hp <= 0 and target not in self.defeated_pieces:
                self.defeated_pieces.append(target)

    def revive_phase(self):
        for piece in self.pieces:
            if isinstance(piece, Medic):
                self.defeated_pieces = piece.revive(self.board, self.defeated_pieces)

        # すべての攻撃が終わってから defeated_pieces を処理する
        for piece in self.defeated_pieces:
            print(f"{piece.symbol} is defeated!")
            self.remove_piece(piece)

    def end_turn(self):
        self.print_board()

    def check_victory(self):
        ally_pieces = [p for p in self.pieces if p.team == 'ally']
        enemy_pieces = [p for p in self.pieces if p.team == 'enemy']

        # 条件1: 敵の全滅
        if not enemy_pieces:
            return 'A'
        if not ally_pieces:
            return 'B'

        # 条件2: 一番奥のマスに到達
        for piece in self.pieces:
            if piece.team == 'ally' and piece.y == self.board_size - 1:
                return 'A'
            if piece.team == 'enemy' and piece.y == 0:
                return 'B'

        # 条件3: 10ターン終了時にHPの合計が大きい方
        if self.current_turn >= self.max_turns:
            ally_hp = sum(p.hp for p in ally_pieces)
            enemy_hp = sum(p.hp for p in enemy_pieces)
            if ally_hp > enemy_hp:
                return 'A'
            elif enemy_hp > ally_hp:
                return 'B'
            else:
                return 'draw'

        return None

    def print_board(self):
        for row in self.board:
            print(' '.join(row))

    def run_game(self):
        # プレイヤーの選択
        player_type_a = input("Select player type for Player A (human/llm/bot): ").lower()
        player_type_b = input("Select player type for Player B (human/bot): ").lower()

        if player_type_a == 'human':
           self.player_a = HumanPlayer('Player A')
        elif player_type_a == 'llm':
            self.player_a = LLM_Player('LLM Player A')
            print("""
            Set the policy for pieces. If you want to use the default policy,\n
            "mobility_policy : You basically follow the policy of moving forward."\n
            "attack_policy : You basically follow the policy of attacking enemies in the forward direction."\n
            enter 0.
            """)
            self.set_pieces_policy('ally')
        else:
            self.player_a = RandomBot('Bot A')

        if player_type_b == 'human':
            self.player_b = HumanPlayer('Player B')
        else:
            self.player_b = RandomBot('Bot B')

        while True:
            self.play_turn()
            victor = self.check_victory()
            if victor:
                if victor == 'draw':
                    print("The game ends in a draw!")
                else:
                    print(f"{victor.capitalize()} team wins!")
                break

# 人間のプレイヤー
CLIで移動場所・攻撃対象を指定



In [ ]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name

    def select_move(self, piece, possible_moves):
        while True:
            print(f"{self.name}'s {piece.symbol} at ({piece.x}, {piece.y}) can move to the following relative positions:")
            for dx, dy in possible_moves:
                print(f"({dx}, {dy})")
            dx = int(input(f"Where do you want to move {piece.symbol}? (dx) "))
            dy = int(input(f"Where do you want to move {piece.symbol}? (dy) "))
            if (dx, dy) in possible_moves:
                return dx, dy
            else:
                print("Invalid move. Please try again.")

    def receive_information(self, piece, enemy_positions):
        print(f"{self.name}'s {piece.symbol} has discovered the following enemy positions:")
        for pos_x, pos_y in enemy_positions:
            print(f"({pos_x}, {pos_y})")

    def select_attack_target(self, attacker, targets):
        print(f"{self.name}'s {attacker.symbol} can attack the following targets:")
        for target in targets:
            print(f"{target.symbol} at ({target.x}, {target.y})")
        target_x = int(input(f"Which target does {attacker.symbol} attack? (x) "))
        target_y = int(input(f"Which target does {attacker.symbol} attack? (y) "))
        for target in targets:
            if target.x == target_x and target.y == target_y:
                return target
        return None

# ランダムボット
ランダムに移動場所・攻撃対象を指定

In [ ]:
import random

class RandomBot:
    def __init__(self, name):
        self.name = name

    def select_move(self, piece, possible_moves):
        # 動かないことも含めてランダムに選択
        return random.choice(possible_moves)

    def receive_information(self, piece, enemy_positions):
        # 情報を受け取るだけで特に何もしない
        pass

    def select_attack_target(self, attacker, targets):
        # ランダムに攻撃対象を選択
        return random.choice(targets) if targets else None


# LLMのプレイヤー
各駒に行動ポリシーを設定し、駒がそれに基づいた行動を決定する

In [ ]:
!pip install flash_attn==2.5.8 torch==2.3.1 accelerate==0.31.0 transformers==4.41.2

In [ ]:
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

class LLM_Player:
    def __init__(self, name):
        self.name = name
        torch.random.manual_seed(0)

    def generate_text(self, messages):
        torch.cuda.empty_cache()
        tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
        model = AutoModelForCausalLM.from_pretrained(
            "microsoft/Phi-3-mini-4k-instruct",
            device_map="cuda",
            torch_dtype="auto",
            trust_remote_code=True,
        )
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
        )
        generation_args = {"max_new_tokens": 500, "return_full_text": False, "temperature": 0.0, "do_sample": False, }

        output = pipe(messages,**generation_args)
        assistant_response = output[0]['generated_text']
        print(assistant_response)
        return assistant_response

    def select_move(self, piece, possible_moves):
        selected = []
        print(f"{self.name}'s {piece.symbol} at ({piece.x}, {piece.y}) can move to the following relative positions:")
        context = ""
        num = 1
        for dx, dy in possible_moves:
            selected.append([dx,dy])
            context += f"\n[{num}]{dy} spaces forward and {dx} spaces to the left."
            num += 1
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant that provides a clear answer. You basically follow the policy of moving forward. You must only answer using a number."},
            {"role": "user", "content": f"[1]1 spaces forward and 0 spaces to the left.\n[2]2 spaces forward and 0 spaces to the left.\n Which option should I choose?"},
            {"role": "assistant", "content": "If I must follow the policy I should choose the option that is moving more forward. Therefore, I choose [2]"},
            {"role": "user", "content": f"{context} Which option should I choose?"},
        ]
        print(context)
        while True:
            answer = self.generate_text(messages)

            # 正規表現パターン
            pattern = r'\[(\d+)\]'
            answer = int(re.findall(pattern, answer)[0])

            dx = selected[answer-1][0]
            dy = selected[answer-1][1]

            if (dx, dy) in possible_moves:
                return dx, dy
            else:
                print("Invalid move. Please try again.")

    def receive_information(self, piece, enemy_positions):
        print(f"{self.name}'s {piece.symbol} has discovered the following enemy positions:")
        for pos_x, pos_y in enemy_positions:
            print(f"({pos_x}, {pos_y})")

    def select_attack_target(self, attacker, targets):
        selected = []
        print(f"{attacker.symbol} can attack the following targets:")
        context = ""
        num = 1
        for target in targets:
            selected.append([target.x, target.y])
            context += f"\n[{num}]{target.x-attacker.x} spaces formard and {target.y-attacker.y} spaces to the left."
            num += 1
        message = [
            {"role": "system", "content": "You are a helpful AI assistant that provides a clear answer. You basically follow the policy of attacking enemies in the forward direction. You must only answer using a number."},
            {"role": "user", "content": f"[1]0 spaces formard and 1 spaces to the left.\n[2]:2 spaces formard and 0 spaces to the left.\n.\n Which option should I choose?"},
            {"role": "assistant", "content": "I must follow the policy. Therefore, I choose [2]"},
            {"role": "user", "content": f"{context} Which option should I choose?"},
        ]
        print(context)
        while True:
            if len(selected) > 1:
                answer = self.generate_text(message)

                # 正規表現パターン
                pattern = r'\[(\d+)\]'
                answer = int(re.findall(pattern, answer)[0])
            else:
                answer = 1

            target_x = selected[answer-1][0]
            target_y = selected[answer-1][1]

            for target in targets:
                if target.x == target_x and target.y == target_y:
                    return target
            return None

# ゲーム開始

In [ ]:
0# 既存の駒クラス（Scout, Medic, HeavyInfantry, Communicator）をここに配置

# ゲームの初期化と実行
game = Game()

# 駒の配置（例）
game.add_piece(Scout(3, 0, 'ally'))
game.add_piece(Medic(2, 0, 'ally'))
game.add_piece(HeavyInfantry(1, 0, 'ally'))
game.add_piece(Communicator(0, 0, 'ally'))

game.add_piece(Scout(5, 8, 'enemy'))
game.add_piece(Medic(6, 8, 'enemy'))
game.add_piece(HeavyInfantry(7, 8, 'enemy'))
game.add_piece(Communicator(8, 8, 'enemy'))

game.run_game()

Select player type for Player A (human/llm/bot): llm
Select player type for Player B (human/bot): bot

            Set the policy for pieces. If you want to use the default policy,

            "mobility_policy : You basically follow the policy of moving forward."

            "attack_policy : You basically follow the policy of attacking enemies in the forward direction."

            enter 0.
            
Input the mobility polisy of S.0
Input the attack polisy of S.0
Input the mobility polisy of M.0
Input the attack polisy of M.0
Input the mobility polisy of H.0
Input the attack polisy of H.0
Input the mobility polisy of C.0
Input the attack polisy of C.0

--- Turn 1 ---
LLM Player A's S at (3, 0) can move to the following relative positions:

[1]1 spaces forward and -2 spaces to the left.
[2]1 spaces forward and -1 spaces to the left.
[3]2 spaces forward and -1 spaces to the left.
[4]1 spaces forward and 0 spaces to the left.
[5]2 spaces forward and 0 spaces to the left.
[6]3 spaces

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 To choose the best option, we need to consider both the forward and leftward movements. The goal is to maximize forward movement while minimizing leftward movement.


Option [6] has the highest forward movement (3 spaces) with no leftward movement. This is the best option as it maximizes forward progress without any detriment to the leftward position.


Therefore, I choose [6].
S moved to (3, 3)
LLM Player A's M at (2, 0) can move to the following relative positions:

[1]1 spaces forward and -1 spaces to the left.
[2]1 spaces forward and 0 spaces to the left.
[3]2 spaces forward and 0 spaces to the left.
[4]0 spaces forward and 1 spaces to the left.
[5]1 spaces forward and 1 spaces to the left.
[6]0 spaces forward and 2 spaces to the left.
[7]0 spaces forward and 0 spaces to the left.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 To determine the best option, we need to consider the following constraints:

1. The goal is to move forward as much as possible.
2. Moving to the left is not desirable.
3. The option with the least negative impact on forward movement should be chosen.

Now let's analyze each option:

[1] -1 spaces forward and 0 spaces to the left.
This option moves backward by 1 space, which is not desirable.

[2] 1 spaces forward and 0 spaces to the left.
This option moves forward by 1 space, which is the best option so far.

[3] 0 spaces forward and 1 spaces to the left.
This option moves to the left by 1 space, which is not desirable.

[4] 0 spaces forward and 0 spaces to the left.
This option does not move forward or to the left, which is not desirable.

Considering the constraints, the best option is [2] because it moves forward by 1 space and does not move to the left.
H moved to (0, 1)
LLM Player A's C at (0, 4) can move to the following relative positions:

[1]-2 spaces forward and 0 spaces t

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 To determine the best option, we need to consider the following constraints:

1. The goal is to move forward as much as possible.

2. Moving to the left is not desirable unless it's necessary to avoid an obstacle or to reach a specific goal.

3. The number of spaces moved forward should be maximized.

4. The number of spaces moved to the left should be minimized.


Considering these constraints, let's evaluate each option:

[1] -2 spaces forward and 0 spaces to the left.

[2] -1 spaces forward and 0 spaces to the left.

[3] 1 spaces forward and 0 spaces to the left.

[4] -1 spaces forward and 1 spaces to the left.

[5] 0 spaces forward and 1 spaces to the left.

[6] 1 spaces forward and 1 spaces to the left.

[7] 0 spaces forward and 2 spaces to the left.

[8] 0 spaces forward and 0 spaces to the left.


Option [3] is the best choice as it moves the most forward (1 space) without moving to the left at all.
C moved to (0, 2)
s moved to (7, 4)
m moved to (6, 4)
h moved to (7, 8)
c moved